In [256]:
import cv2
import numpy as np
import glob

In [257]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [258]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [259]:
# points: the 4 points of the region that we picked by hand
def get_perspective_transform(points, height, width):
    source = np.float32(np.array(points))
    destination = np.float32([[0, height], [width, height], [width, 0], [0, 0]])
    perspective_transform = cv2.getPerspectiveTransform(source, destination)
    return perspective_transform


In [260]:
# human_boxes: the points of bounding box of all detected human, output of yolov3
def get_perspective_points(human_boxes, prespective_transform):
    perspective_points = []
    for human in human_boxes:
        points = np.array([[[int(human[0]+(human[2]*0.5)),int(human[1]+human[3])]]], dtype="float32")
        perspective_point = cv2.perspectiveTransform(points, prespective_transform)[0][0]
        perspective_points += [(int(perspective_point[0]), int(perspective_point[1]))]
    return perspective_points

In [261]:
def bird_eye_view(height, width, measured_distance, perspective_points):
    red = (0, 0, 255)
    green = (0, 255, 0)
    white = (200, 200, 200)
    # white picture, 8-bit unsigned integer (0 to 255)
    result_frame = np.zeros((int(height), int(width), 3), np.uint8)
    result_frame[:] = white
    unsafe = []
    safe = []

    if len(measured_distance) != 0:
        for pair in measured_distance:
            if pair[0] <= 2: # the unsafe distance is 2m
                if (pair[1] not in unsafe) and (pair[1] not in safe):
                    unsafe += [pair[1]]
                if (pair[2] not in unsafe) and (pair[2] not in safe):
                    unsafe += [pair[2]]
                result_frame = cv2.line(result_frame, (int(pair[1][0]), int(pair[1][1])), (int(pair[2][0]), int(pair[2][1])), red, 2)

    if len(perspective_points) != 0:   
        for human in perspective_points:
            if human not in unsafe:
                safe += [human]
    
    for plot in safe:
        result_frame = cv2.circle(result_frame, (int(plot[0]), int(plot[1])), 5, green, 10)
    for plot in unsafe:
        result_frame = cv2.circle(result_frame, (int(plot[0]), int(plot[1])), 5, red, 10)
        
    return result_frame, len(unsafe)

In [262]:
def get_pixel_scale_of_2m(target_reference, perspective_matrix):
    ''' target_reference[0]: shoulder_to_height_ratio of the target
        target_reference[1]: x-coordinate of the target on its bottom left
        target_reference[2]: y-coordinate of the target on its bottom left
        target_reference[3]: width of the target
        target_reference[4]: height of the target
        output: safe distance of 2m's representation in bird view perspective.
    '''
    shoulder_to_height_ratio = target_reference[0]
    x = target_reference[1]
    y = target_reference[2]
    width = target_reference[3]
    height = target_reference[4]
    assumed_real_height = 1.70
    absolute_scale_shoulder = (assumed_real_height / height) * width
    points = np.float32(np.array([[[x, y],[x+width, y]]]))
    print(points)
    perspective_point1 = cv2.perspectiveTransform(points, perspective_matrix)[0][0]
    perspective_point2 = cv2.perspectiveTransform(points, perspective_matrix)[0][1]
    dis_x = abs(perspective_point2[0]-perspective_point1[0])
    dis_y = abs(perspective_point2[1]-perspective_point1[1])
    distance = int(np.sqrt(((dis_x)**2) + ((dis_y)**2)))
    pixel_distance_of_2m = int(round(distance / absolute_scale_shoulder * 2))
    return pixel_distance_of_2m

In [263]:
def measure_distance(safety_distance, perspective_points):
    measured_distance = []
    for human1 in perspective_points:
        for human2 in perspective_points:
            if human1 != human2:
                dis_x = abs(human2[0]-human1[0])
                dis_y = abs(human2[1]-human1[1])
                distance = float((np.sqrt(((dis_x)**2) + ((dis_y)**2))/safety_distance) * 2)
                measured_distance += [(distance, human1, human2)]        
    return measured_distance

In [264]:
def social_distancing_view(frame, perspective_transform, boxes, safety_distance):
    """
    boxes: list of [x of left border, y of top border, width, height] in original video
    """
    red = (0, 0, 255)
    green = (0, 255, 0)

    # draw green box for everyone
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), green, 2)

    for i in range(len(boxes)):
        for j in range(len(boxes)):
            if j != i:
                x1, y1, w1, h1 = boxes[i]
                x2, y2, w2, h2 = boxes[j]
                points1 = np.array([[[int(x1 + (w1 * 0.5)), int(y1 + h1)]]], dtype="float32")
                points2 = np.array([[[int(x2 + (w2 * 0.5)), int(y2 + h2)]]], dtype="float32")
                perspective_point1 = cv2.perspectiveTransform(points1, perspective_transform)[0][0]
                perspective_point2 = cv2.perspectiveTransform(points2, perspective_transform)[0][0]
                perspective_points1 = (int(perspective_point1[0]), int(perspective_point1[1]))
                perspective_points2 = (int(perspective_point2[0]), int(perspective_point2[1]))
                if len(measure_distance(safety_distance, [perspective_points1, perspective_points2])) != 0:
                    distance = measure_distance(safety_distance, [perspective_points1, perspective_points2])[0][0]

                if distance <= 2:   # unsafe
                    # box for human 1
                    frame = cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), red, 2)
                    # box for human 2
                    frame = cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), red, 2)
                    # line between human 1 and human 2
                    frame = cv2.line(frame, (int(x1 + w1 / 2), int(y1 + h1 / 2)), (int(x2 + w2 / 2), int(y2 + h2 / 2)), red, 2)


    return frame

In [265]:
# Acknowledge to Nayak, S. (2018). Deep Learning based Object Detection using YOLOv3 with OpenCV ( Python / C++). 
# Retrieved 18 December 2020, from https://www.learnopencv.com/deep-learning-based-object-detection-using-yolov3-with-opencv-python-c/
# Learned the usage and the implementation YOLOv3 on object detection from this website. 
def get_social_distance_detected_video(picked_pts, video_path):
    model = '/content/drive/MyDrive/CSC420/project/yolov3.weights'
    config = '/content/drive/MyDrive/CSC420/project/yolov3.cfg'
    video = cv2.VideoCapture(video_path)
    net_layer = cv2.dnn.readNetFromDarknet(config, model)
    layersNames = net_layer.getLayerNames()
    outputlayers_Names = [layersNames[i[0] - 1] for i in net_layer.getUnconnectedOutLayers()]
    net = cv2.dnn.readNetFromDarknet(config, model)
    hasFrame = True
    frame_num = 0
    shoulder_to_height_ratio = []
    while (hasFrame):
        (hasFrame, frame) = video.read()
        if not hasFrame:
            break
        height = []
        frames = []
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        predictions_array = net.forward(outputlayers_Names)
        boxes = []
        framesID = []
        frame_height, frame_width = frame.shape[0],frame.shape[1]
        for output in predictions_array:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if classID == 0 and confidence > 0.8:
                    box = detection[0:4] * np.array([frame_width, frame_height, frame_width, frame_height])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(round((centerX - (width / 2))))
                    y = int(round((centerY + (height / 2))))
                    shoulder_to_height_ratio.append((width/height, x, y, width, height))
        frame_num += 1
        print(frame_num)
    human_count = len(shoulder_to_height_ratio)
    shoulder_to_height_ratio.sort()
    target_reference = shoulder_to_height_ratio[human_count//2]
    # target_reference = (0.45652173913043476, 1106, 142, 63, 138)
    # picked_pts = [[1800, 100], [1800, 900], [100, 900], [100, 100]]
    # transform_matrix = get_perspective_transform(picked_pts, 1080, 1920)
    transform_matrix = get_perspective_transform(picked_pts, frame_height, frame_width)
    safety_distance = get_pixel_scale_of_2m(target_reference, transform_matrix)
    print(safety_distance)

    video = cv2.VideoCapture(video_path)
    net_layer = cv2.dnn.readNetFromDarknet(config, model)
    layersNames = net_layer.getLayerNames()
    outputlayers_Names = [layersNames[i[0] - 1] for i in net_layer.getUnconnectedOutLayers()]
    net = cv2.dnn.readNetFromDarknet(config, model)
    hasFrame = True
    frame_num = 0
    while (hasFrame):
        (hasFrame, frame) = video.read()
        if not hasFrame:
            break
        height = []
        frames = []
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        predictions_array = net.forward(outputlayers_Names)
        boxes = []
        framesID = []
        confidences = []
        frame_height, frame_width = frame.shape[0],frame.shape[1]
        for output in predictions_array:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if classID == 0 and confidence > 0.8:
                    box = detection[0:4] * np.array([frame_width, frame_height, frame_width, frame_height])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(round((centerX - (width / 2))))
                    y = int(round((centerY - (height / 2))))
                    boxes += [[x, y, int(width), int(height)]]
                    confidences += [float(confidence)]
        frame_num += 1
        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
        boxes1 = []
        for i in indices:
            i = i[0]
            box = boxes[i]
            boxes1 += [box]
            

        perspective_points = get_perspective_points(boxes, transform_matrix)
        measured_distance = measure_distance(safety_distance, perspective_points)
        bird_eye_view_image, num_unsafe = bird_eye_view(frame_height, frame_width, measured_distance, perspective_points)
        for point in picked_pts:
            frame = cv2.circle(frame, (int(point[0]), int(point[1])), 5, (200, 200, 200), 10)
        frame = cv2.line(frame, (int(picked_pts[0][0]), int(picked_pts[0][1])), (int(picked_pts[1][0]), int(picked_pts[1][1])), (200, 200, 200), 2)
        frame = cv2.line(frame, (int(picked_pts[1][0]), int(picked_pts[1][1])), (int(picked_pts[2][0]), int(picked_pts[2][1])), (200, 200, 200), 2)
        frame = cv2.line(frame, (int(picked_pts[2][0]), int(picked_pts[2][1])), (int(picked_pts[3][0]), int(picked_pts[3][1])), (200, 200, 200), 2)
        frame = cv2.line(frame, (int(picked_pts[3][0]), int(picked_pts[3][1])), (int(picked_pts[0][0]), int(picked_pts[0][1])), (200, 200, 200), 2)
        social_distance_view_image = social_distancing_view(frame, transform_matrix, boxes1, safety_distance)
        if not cv2.imwrite('/content/drive/MyDrive/CSC420/project/bird_eye_view/frame%d.jpg' % frame_num, bird_eye_view_image):
            print("Could not write image")
        if not cv2.imwrite('/content/drive/MyDrive/CSC420/project/social_distance_view/frame%d.jpg' % frame_num, social_distance_view_image):
            print("Could not write image")
        print('%d frame done' % frame_num)


In [266]:
ref = get_social_distance_detected_video([[1800, 100], [1800, 900], [100, 900], [100, 100]], '/content/drive/MyDrive/CSC420/project/test.mp4')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [267]:
def create_video(path):
    img_list = []
    count = 0
    for filename in glob.glob(path + '/*.jpg'):
        count += 1
    for i in range(count):
        index = i+1
        img = cv2.imread(path + '/frame' + str(index) +'.jpg')
        img_list += [img]

    height,width,layers = img_list[0].shape
    out = cv2.VideoWriter(path + '/out.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, (width, height))
    for img in img_list:
        out.write(img)
    out.release()

In [268]:
create_video('/content/drive/MyDrive/CSC420/project/social_distance_view')

In [269]:
create_video('/content/drive/MyDrive/CSC420/project/bird_eye_view')